### G. Последний клик перед отлётом (30 баллов)

### Ограничения
- **Время:** 10 секунд
- **Память:** 244.14 Мб
- **Ввод:** CSV-файл с данными
- **Вывод:** `output.txt`

### Описание задачи
---

Работа в IT оказалась для дрона невыносимой. Постоянные стендапы, странные гифки в чатах и непредсказуемые пайплайны — всё это оказалось далёко от логичного и рационального мира его родной планеты. Но одна вещь по-настоящему впечатлила его: **люди обожают игры**.

Проект оказался успешным: пользователи заходили регулярно, смотрели рекламу, совершали покупки, сражались, проходили квесты и поднимались по уровням. Теперь пришло время подвести итоги: **сможет ли он заработать достаточно, чтобы отремонтировать корабль и улететь домой?**

Для этого дрон собрал огромную таблицу history.csv с пользовательской статистикой за всё время вплоть до **апреля 2025 года**. На основе этих данных ему нужно понять:

* какие игроки будут **активны в мае 2025 года**;
* сколько денег он получит в мае, включая:

  * выручку от **внутриигровых покупок**;
  * выручку от **рекламы**.

---

### 📥 Формат ввода

CSV-файл содержит следующую информацию:

| Поле                        | Описание                                                                   |
| --------------------------- | -------------------------------------------------------------------------- |
| `id`                        | Уникальный идентификатор игрока                                            |
| `logins_current_month`      | Количество дней, когда игрок заходил в игру за последний календарный месяц |
| `country`                   | Страна игрока                                                              |
| `traffic_type`              | Источник трафика (естественный или рекламный)                              |
| `platform`                  | Платформа (Android или iOS)                                                |
| `registration_date`         | Дата регистрации                                                           |
| `ads_shown_current_month`   | Кол-во показов рекламы за последний месяц                                  |
| `ads_revenue_current_month` | Выручка от рекламы за последний месяц                                      |
| `ads_shown_current_day`     | Показов рекламы в последний игровой день                                   |
| `ads_revenue_current_day`   | Выручка от рекламы в последний день                                        |
| `revenue_current_month`     | Выручка от внутриигровых покупок за месяц                                  |
| `games_pvp`                 | Кол-во PvP-сражений за месяц                                               |
| `wins_pvp`                  | Победы в PvP за месяц                                                      |
| `quests`                    | Выполненные задания за месяц                                               |
| `hard_quests`               | Сложные квесты за месяц                                                    |
| `current_passed_level`      | Последний пройденный уровень                                               |
| `offers`                    | Кол-во предложений                                                         |
| `currency_i`                | Внутриигровая валюта №i                                                    |
| `current_avg_ping`          | Средний пинг игрока                                                        |
| `current_month`             | Месяц, к которому относятся данные                                         |

---

### 📤 Формат вывода

Выходной файл — CSV с **тремя колонками**:

| Колонка              | Описание                                                                                   |
| -------------------- | ------------------------------------------------------------------------------------------ |
| `id`                 | Идентификатор пользователя                                                                 |
| `revenue_next_month` | Предсказанная выручка от пользователя в мае (покупки + реклама), **неотрицательное число** |
| `is_active`          | `1`, если игрок **будет активен в мае** (зайдёт хотя бы раз); иначе `0`                    |

---

### 🧮 Метрики и оценка

#### Классификация (`is_active`)

Вычисляется \$F\_1\$-мера **относительно положительного класса** (\$1\$).

#### Регрессия (`revenue_next_month`)

Считается **log-RMSE** — логарифмическая среднеквадратичная ошибка среди активных пользователей:

```python
import numpy as np
from sklearn.metrics import mean_squared_error

def log_rmse(y_true, y_pred):
    return mean_squared_error(np.log1p(y_true), np.log1p(y_pred), squared=False)
```

#### Финальный скор

Общая оценка рассчитывается по формуле:

$$
\text{Score} = 15 \cdot F_1 + \max(15 - 20 \cdot \text{log\_rmse}, 0)
$$

---

📌 Максимальный балл: **30**
— До **15 баллов** за классификацию
— До **15 баллов** за регрессию

---


In [216]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

## Classiication

In [217]:
df = pd.read_csv('history.csv')

In [ ]:
df["traffic_type"]

In [219]:
"""
#traffic_type
#platform
registration_date
current_month
"""

'\n#traffic_type\n#platform\nregistration_date\ncurrent_month\n'

In [220]:
df = df.sort_values('id')
df["IOS"] = df["platform"].apply(lambda x: 1 if x == "iOS" else 0)
df["organic"] = df["traffic_type"].apply(lambda x: 1 if x == "organic" else 0)
df.drop(columns=["platform", "traffic_type"], inplace=True)

In [221]:
df['current_month'] = pd.to_datetime(df['current_month'])
df['next_month'] = df['current_month'] + pd.DateOffset(months=1)
id_month_set = set(df[['id', 'current_month']].itertuples(index=False, name=None))

def check_online(row):
        return int((row['id'], row['next_month']) in id_month_set)

df['isOnlineNextMonth'] = df.apply(check_online, axis=1)


In [222]:
df['registration_date'] = pd.to_datetime(df['registration_date'], format='%Y-%m-%d')
may_1_2025 = pd.to_datetime('2025-05-01')
df['days_since_registration'] = (may_1_2025 - df['registration_date']).dt.days
df.drop(columns=["registration_date"], inplace=True)

In [223]:
df['isOnlineNextMonth_filled'] = df['isOnlineNextMonth'].fillna(-1)  # чтобы None не мешал
df = df.sort_values(
    by=['id', 'isOnlineNextMonth_filled', 'current_month'],
    ascending=[True, False, False]
)

df = df.drop(columns=['isOnlineNextMonth_filled']).groupby('id', as_index=False).first()


In [224]:
pos_df = df[df['isOnlineNextMonth'] == 1]
neg_df = df[df['isOnlineNextMonth'] == 0].sample(frac=1, random_state=42)
train_df = pd.concat([pos_df, neg_df], ignore_index=True)


In [225]:
target = 'isOnlineNextMonth'
ignore_cols = ['id', 'current_month', 'next_month', target]

# Признаки
X = train_df.drop(columns=ignore_cols)
y = train_df[target]

# Автоматически определим категориальные признаки (если есть строковые)
cat_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

In [226]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [227]:
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    eval_metric='F1',
    verbose=100,
    random_seed=42
)

model.fit(X_train, y_train, cat_features=cat_features, eval_set=(X_test, y_test), early_stopping_rounds=50)

0:	learn: 0.6937447	test: 0.6959132	best: 0.6959132 (0)	total: 13.8ms	remaining: 13.8s
100:	learn: 0.7176552	test: 0.7193589	best: 0.7193589 (100)	total: 1.25s	remaining: 11.2s
200:	learn: 0.7282743	test: 0.7289174	best: 0.7290633 (197)	total: 2.38s	remaining: 9.47s
300:	learn: 0.7365254	test: 0.7373452	best: 0.7373452 (300)	total: 3.43s	remaining: 7.96s
400:	learn: 0.7433666	test: 0.7430876	best: 0.7431041 (397)	total: 4.47s	remaining: 6.68s
500:	learn: 0.7474321	test: 0.7434864	best: 0.7445339 (466)	total: 5.61s	remaining: 5.59s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.7445339471
bestIteration = 466

Shrink model to first 467 iterations.


In [228]:
y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[18007   750]
 [ 2802  5176]]
              precision    recall  f1-score   support

           0       0.87      0.96      0.91     18757
           1       0.87      0.65      0.74      7978

    accuracy                           0.87     26735
   macro avg       0.87      0.80      0.83     26735
weighted avg       0.87      0.87      0.86     26735



In [229]:
train_ids = set(train_df['id'])
full_ids = set(df['id'])
predict_ids = full_ids - train_ids

df_to_predict = df[df['id'].isin(predict_ids)].copy()
X_pred = df_to_predict.drop(columns=['id', 'current_month', 'isOnlineNextMonth'], errors='ignore')
df_to_predict['is_active'] = model.predict(X_pred)

df_train_active = train_df.copy()
df_train_active['is_active'] = df_train_active['isOnlineNextMonth']

final_df = pd.concat([df_train_active, df_to_predict], ignore_index=True)

No objects info loaded


In [230]:
final_df = final_df.sort_values('id')
final_df

,id,logins_current_month,country,ads_shown_current_month,ads_revenue_current_month,ads_shown_current_day,ads_revenue_current_day,revenue_current_month,games_pvp,wins_pvp,...,currency_6,currency_7,current_avg_ping,current_month,IOS,organic,next_month,isOnlineNextMonth,days_since_registration,is_active
83939,1,1,81,1,0.867439,0,0.0,8.87251,18,7,...,0,0,0.263144,2024-09-01,1,0,2024-10-01,0,322,0
95222,2,1,193,0,0.000000,0,0.0,0.00000,0,0,...,0,0,0.000000,2024-10-01,0,0,2024-11-01,0,334,0
0,3,7,180,0,0.000000,0,0.0,0.00000,26,20,...,0,0,0.430013,2024-12-01,0,0,2025-01-01,1,140,1
1,4,22,106,11,0.077312,0,0.0,0.00000,266,140,...,0,0,0.396413,2024-12-01,0,0,2025-01-01,1,147,1
124299,5,1,208,0,0.000000,0,0.0,0.00000,3,3,...,0,0,0.158504,2025-01-01,0,0,2025-02-01,0,118,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39890,133670,7,81,31,2.174221,0,0.0,0.00000,51,34,...,0,0,0.257087,2025-01-01,1,0,2025-02-01,1,102,1
122899,133671,1,173,0,0.000000,0,0.0,0.00000,0,0,...,0,0,0.000000,2025-02-01,0,0,2025-03-01,0,70,0
39891,133672,6,173,4,0.018048,0,0.0,0.00000,93,57,...,0,0,0.314705,2024-12-01,0,0,2025-01-01,1,126,1
108469,133673,14,49,32,0.042859,0,0.0,0.00000,161,90,...,0,0,0.359385,2024-10-01,0,0,2024-11-01,0,202,0


## Regression

In [235]:
ids_active = final_df.loc[final_df['is_active'] == 1, 'id']

In [339]:
df = pd.read_csv('history.csv')

df = df.sort_values('id')
df["IOS"] = df["platform"].apply(lambda x: 1 if x == "iOS" else 0)
df["organic"] = df["traffic_type"].apply(lambda x: 1 if x == "organic" else 0)
df.drop(columns=["platform", "traffic_type"], inplace=True)

df['registration_date'] = pd.to_datetime(df['registration_date'], format='%Y-%m-%d')
may_1_2025 = pd.to_datetime('2025-05-01')
df['days_since_registration'] = (may_1_2025 - df['registration_date']).dt.days
df.drop(columns=["registration_date"], inplace=True)


In [340]:
df['sum'] = df['revenue_current_month'] + df['ads_revenue_current_month']
df.drop(columns=['revenue_current_month', 'ads_revenue_current_month', 'ads_shown_current_month', 'ads_shown_current_day', 'ads_revenue_current_day'], inplace=True)

## Create submission

In [349]:
df_s = pd.read_csv('history.csv')
df_s['is_active'] = 0
df_s['next_month_revenue'] = 0.0
df_sorted = df_s.sort_values('id', ascending=True)
df_unique = df_sorted.drop_duplicates(subset='id', keep='first')
submission = df_unique[['id', 'next_month_revenue', 'is_active']]


In [ ]:
final_df['id'] = final_df['id'].astype(str)
submission['id'] = submission['id'].astype(str)

id_to_active = dict(zip(final_df['id'], final_df['is_active']))
submission['is_active'] = submission['id'].map(id_to_active)

/var/folders/v6/28k1t0ds7wsdzssl417x6kww0000gn/T/ipykernel_7154/1199957629.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['id'] = submission['id'].astype(str)
/var/folders/v6/28k1t0ds7wsdzssl417x6kww0000gn/T/ipykernel_7154/1199957629.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['is_active'] = submission['id'].map(id_to_active)


In [355]:
submission

,id,next_month_revenue,is_active
14996,1,0.0,0
60701,2,0.0,0
111702,3,10.0,1
109052,4,10.0,1
122443,5,0.0,0
...,...,...,...
137223,133670,10.0,1
146553,133671,0.0,0
143875,133672,10.0,1
52746,133673,0.0,0


## Save submission

In [356]:
submission.to_csv('submission.csv', index=False)